In [19]:
import osmnx as ox
import pandas as pd
from pathlib import Path
import numpy as np

import src.wrangle_bbr as wrangle

In [20]:
DATA_DIR = Path("data")
df = pd.read_feather(DATA_DIR / "select_bbr2.feather")
df.fillna(value=np.nan, inplace=True)
wrangle.clean_columns(df)
dtypes = {"varmeinstallation": "int8", 
          "opvarmningsmiddel": "float",
          }
df = df.astype(dtypes)
df = df[~df["koordinat"].isin(['POINT(0 0)', 'POINT(0 0.5)'])]
# make into category
df["bygningens_anvendelse"] = df["bygningens_anvendelse"].astype("category")


ImportError: Missing optional dependency 'pyarrow'.  Use pip or conda to install pyarrow.

In [3]:

place = "Thisted Municipality"
G = ox.graph_from_place(place, network_type="drive")
Gp = ox.project_graph(G)

In [5]:
points = ox.utils_geo.sample_points(ox.get_undirected(Gp), n=100)
X = points.x.values
Y = points.y.values
X0 = X.mean()
Y0 = Y.mean()

In [17]:
Y.min()

6281593.249334324